# Análise de sentimento em avaliações de Filmes (IMDb)

Neste notebook vamos treinar um modelo estatístico para realizar uma análise de sentimento em avaliações de filmes. Depois de treinarmos o modelo vamos salva-lo para depois utilizarmos ele em um DAG do airflow.

Para as etapas de pré processamento vamos utilizar duas bibliotecas muito utilizadas para NLP: nltk e spacy

*IMDb: Internet Movie Database

### imports

In [ ]:
import pandas as pd

import nltk
import spacy
import re

from unicodedata import normalize

### Leitura do Dataframe

In [ ]:
df = #TODO: leitura do csv
df.head(10)

### Contagem dos sentimentos

In [ ]:
#TODO: contar a quantidade de avaliações por classe

### Preparação dos dados 

A primeira etapa que vamos realizar é a obtenção das stopwords no idioma português. As stopwords mais tarde serão removidas do dataset pois elas não contém informações relevantes para a análise.

In [ ]:
!python -m spacy download pt --user

nlp = spacy.load('pt')
nltk.download('stopwords')

stopwordsnltk = nltk.corpus.stopwords.words('portuguese')

stopwordsspacy = spacy.lang.pt.stop_words.STOP_WORDS

stopwords = set(stopwordsnltk + list(stopwordsspacy))



In [ ]:
stopwords

Vamos elimar a coluna que contém a avaliação em inglês

In [ ]:
#TODO: remover a coluna 'text_en' do dataset

Nesse passo realizamos a codificação da coluna 'sentiment'

In [ ]:
#TODO: codificar com inteiros (0 ou 1) a coluna 'sentiment', 0 para 'neg' e 1 para 'pos'. 
#Escrever em uma outra coluna com o nome 'encod_sent'

In [ ]:
encoding_target(df)

Vamos também retirar outros fatores que introduzem ruídos no nosso dataset como acentos ou se a palavra contém letras maiúsculas ou minúsculas.

In [ ]:
def remover_acentos(texto):
    return normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII').lower()

def remover_acentos_pontuacoes(df):
    df['text_pt'] = df.text_pt.apply(lambda texto: remover_acentos(texto))
    df['text_pt'] = df.text_pt.apply(lambda texto: re.sub(u'[^a-z]', ' ', texto))
    df['text_pt'] = df.text_pt.apply(lambda texto: re.sub(u'\s+', ' ', texto))
    return df

In [ ]:
remover_acentos_pontuacoes(df)

In [ ]:
stopwords = [remover_acentos(stop) for stop in stopwords]

In [ ]:
def remove_stop_words(df):
    df['text_pt_w_sw'] = df.text_pt.apply(lambda texto: ' '.join(p for p in texto.split() if p not in stopwords))
    return df

In [ ]:
remove_stop_words(df)

Aqui vamos gerar vetores de tamanho 5000 baseados em tf-idf para cada uma das avaliações 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1,1), use_idf=True, max_features=5000)
text_vect = vectorizer.fit_transform(df['text_pt_w_sw'])

### Modelagem

Vamos treinar dois modelos e escolher aquele com a melhor pontuação de acurácia

*Você pode experimentar com outros modelos vistos durante o curso e treina-los, será que é possível alcançar acurácia maior?

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = #TODO: Separar em datasets de treino e teste, com 20% para teste e estratificado 
#com base no tipo de avaliação, coluna 'encod_sent'

#usar o parâmetro 'stratify' passando a coluna 'encod_sent'

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(X_train, y_train)
y_predict_naive = clf.predict(X_test)
acc = accuracy_score(y_predict_naive, y_test)

print(acc)

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = #TODO: Treinar o modelo de regressão logística e mostrar a sua acurácia

Qual foi o modelo com melhor resultado?

Abaixo vemos uma matriz de confusão para termos uma ideia mais ampla sobre a performance do modelo escolhido

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, #TODO: passar o array de valores 'preditos' pelo modelo escolhido como parâmetro))

### Persistindo o modelo final

In [ ]:
from joblib import dump
dump(#TODO: passar o modelo escolhido como parâmetro, 'modeloFinal4linux.joblib') 